# Class 4

## The Numpy Version

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Function to read vectors from a file
def read_vectors(filename):
    vectors = np.loadtxt(filename, dtype=int)
    return vectors.T  # Transpose to make columns as vectors

# Load vectors from a file (replace 'filename.txt' with the actual file path)
vectors = read_vectors(
    'latt_n5_Mersene11.txt'
    # 'latt_n5_Mersene19.txt'
    )

print(vectors)

### Task 1: Length of a vector

In [ ]:
def compute_l2_norm_squared(vectors):
    return np.sum(vectors**2, axis=1)

l2_norms_squared = compute_l2_norm_squared(vectors)
l2_norms_df = pd.DataFrame(l2_norms_squared, columns=['L2 Norm Squared']).T
print("L2 Norm Squared for Each Vector:\n", l2_norms_df)

### Task 2: Inner product of vectors

In [ ]:
def compute_inner_products(vectors):
    n = vectors.shape[1]
    inner_products = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(i, n):
            inner_product = np.dot(vectors[i], vectors[j])
            inner_products[i, j] = inner_product
            inner_products[j, i] = inner_product  # Symmetric matrix
    return inner_products

inner_products = compute_inner_products(vectors)
inner_products_df = pd.DataFrame(inner_products, columns=[f"b{i+1}" for i in range(inner_products.shape[1])],
                                 index=[f"b{i+1}" for i in range(inner_products.shape[0])])
print("\nInner Products Matrix:\n", inner_products_df)

### Task 3: Gram-schmidt orthogonalisation (GSO) vector

In [ ]:
def gram_schmidt(vectors):
    vectors = np.array(vectors, dtype=float)  # Ensure vectors is a NumPy array
    n, m = vectors.shape  # n is the number of vectors (rows), m is the dimension of each vector
    gs_vectors = np.zeros((n, m), dtype=float)  # Initialize array for orthogonalized vectors

    for i in range(n):
        vi = vectors[i].copy()  # Copy current vector row

        for j in range(i):
            # Project vi onto gs_vectors[j] (using row-wise indexing)
            projection = np.dot(vi, gs_vectors[j]) / np.dot(gs_vectors[j], gs_vectors[j]) if np.dot(gs_vectors[j], gs_vectors[j]) != 0 else 0
            vi -= projection * gs_vectors[j]  # Subtract projection from vi

        gs_vectors[i] = vi  # Store the orthogonalized vector

    return gs_vectors

gs_vectors = gram_schmidt(vectors)
gs_vectors_df = pd.DataFrame(gs_vectors, columns=[f"b*{i+1}" for i in range(gs_vectors.shape[1])])
print("\nGram-Schmidt Orthogonalized Vectors:\n", gs_vectors_df)

### Task 4: General projection of vectors

In [ ]:
def compute_projections(vectors, gs_vectors):
    n = vectors.shape[1]
    projections = {}
    for k in range(n):
        for i in range(k):
            projection = gs_vectors[k].copy()
            for j in range(i):
                mu = np.dot(vectors[k], gs_vectors[j]) / np.dot(gs_vectors[j], gs_vectors[j])
                projection -= mu * gs_vectors[j]
            projections[(i, k)] = projection
    return projections

projections = compute_projections(vectors, gs_vectors)
for (i, k), proj in projections.items():
    print(f"Projection of vector {k} with respect to vector {i}:\n", proj)

## Without Numpy

In [1]:

def read_vectors(filename):
    vectors = []
    with open(filename, 'r') as file:
        for line in file:
            # Split line into integers and add as a column vector
            vector = [int(num) for num in line.strip().split()]
            vectors.append(vector)
    # Transpose the list of lists to make columns as vectors
    return [list(col) for col in zip(*vectors)]

vectors = read_vectors(
    'latt_n5_Mersene11.txt'
    # 'latt_n5_Mersene19.txt'
    )

for i in vectors:
    print(i)

[1330, 0, 0, 0, 0]
[454, 544, 0, 0, 0]
[1637, 107, 165, 0, 0]
[1624, 248, 1921, 1901, 0]
[1272, 1939, 429, 686, 1733]


### Task 1: Length of a vector

In [2]:
def norm_square(vector):
    return sum([x * x for x in vector])

def compute_l2_norm_squared(vectors):
    l2_norms_squared = []
    for vector in vectors:
        l2_norms_squared.append(norm_square(vector))
    return l2_norms_squared

def print_l2_norms(l2_norms_squared):
    print("\nL2 Norm Squared for Each Vector")
    print("-" * 35)
    for i, norm in enumerate(l2_norms_squared, 1):
        print(f"b{i}: {norm:.2f}")

In [3]:
l2_norms_squared = compute_l2_norm_squared(vectors)
print_l2_norms(l2_norms_squared)


L2 Norm Squared for Each Vector
-----------------------------------
b1: 1768900.00
b2: 502052.00
b3: 2718443.00
b4: 10002922.00
b5: 9035631.00


### Task 2: Inner pro duct of vectors

In [4]:
def inner_product(v1, v2):
    return sum([i * j for i, j in zip(v1, v2)])  # Use zip to pair corresponding elements

def compute_inner_products(vectors):
    n = len(vectors)
    inner_products_matrix = [[0] * n for _ in range(n)]  # Renamed to avoid conflict
    for i in range(n):
        for j in range(i, n):
            # Call the helper function inner_product with vectors[i] and vectors[j]
            inner_product_value = inner_product(vectors[i], vectors[j])
            inner_products_matrix[i][j] = inner_product_value
            inner_products_matrix[j][i] = inner_product_value  # Symmetric matrix
    return inner_products_matrix

def print_inner_products(inner_products):
    print("\nInner Products Matrix")
    print("-" * 50)
    # Create headers
    headers = [f"b{i+1}" for i in range(len(inner_products))]
    print("     " + " | ".join(header.center(8) for header in headers))
    print("-" * 50)

    # Print each row
    for i, row in enumerate(inner_products):
        row_str = " | ".join(f"{value:.2f}".center(8) for value in row)
        print(f"b{i+1}".ljust(5) + row_str)


In [5]:
inner_products = compute_inner_products(vectors)
print_inner_products(inner_products)


Inner Products Matrix
--------------------------------------------------
        b1    |    b2    |    b3    |    b4    |    b5   
--------------------------------------------------
b1   1768900.00 | 603820.00 | 2177210.00 | 2159920.00 | 1691760.00
b2   603820.00 | 502052.00 | 801406.00 | 872208.00 | 1632304.00
b3   2177210.00 | 801406.00 | 2718443.00 | 3001989.00 | 2360522.00
b4   2159920.00 | 872208.00 | 3001989.00 | 10002922.00 | 4674795.00
b5   1691760.00 | 1632304.00 | 2360522.00 | 4674795.00 | 9035631.00


### Task 3: Gram-Schmidt orthogonalisation (GSO) vectors

In [6]:
def gram_schmidt(vectors):
    n = len(vectors)  # Number of vectors
    gs_vectors = [[0] * len(vectors[0]) for _ in range(n)]

    for i in range(n):
        # Start with the current vector
        vi = vectors[i][:]

        for j in range(i):
            # Project vi onto gs_vectors[j]
            dot_product = inner_product(vi, gs_vectors[j])
            gs_norm_squared = norm_square(gs_vectors[j])

            if gs_norm_squared != 0:  # Avoid division by zero
                # Calculate projection and subtract it from vi
                projection = [dot_product / gs_norm_squared * gs_vectors[j][k] for k in range(len(vi))]
                vi = [vi[k] - projection[k] for k in range(len(vi))]

        # After all projections are removed, vi is now orthogonal to previous vectors
        gs_vectors[i] = vi

    return gs_vectors

# Function to print the Gram-Schmidt orthogonalized vectors
def print_original_vectors(vectors):
    print("\nOriginal Vectors in Expected Format")
    print("-" * 50)
    headers = [f"b*{i+1}" for i in range(len(vectors[0]))]
    print("     " + " | ".join(header.center(8) for header in headers))
    print("-" * 50)

    for i, row in enumerate(vectors):
        row_str = " | ".join(f"{value:.2f}".center(8) for value in row)
        print(f"b{i+1}".ljust(5) + row_str)

In [7]:
gs_vectors = gram_schmidt(vectors)
print_original_vectors(gs_vectors)


Original Vectors in Expected Format
--------------------------------------------------
       b*1    |   b*2    |   b*3    |   b*4    |   b*5   
--------------------------------------------------
b1   1330.00  |   0.00   |   0.00   |   0.00   |   0.00  
b2     0.00   |  544.00  |   0.00   |   0.00   |   0.00  
b3     0.00   |   0.00   |  165.00  |   0.00   |   0.00  
b4    -0.00   |   0.00   |   0.00   | 1901.00  |   0.00  
b5     0.00   |   0.00   |   0.00   |   0.00   | 1733.00 


### Task 4: General pro jection of vectors

In [ ]:
def compute_projections(vectors, gs_vectors):
    n = len(vectors)  # Number of vectors (rows)
    projections = {}  # Dictionary to store projections

    for k in range(1, n):  # Start from the second vector (index 1) because the first vector has no predecessors
        for i in range(k):  # Iterate over each predecessor vector up to (but not including) k
            # Calculate the projection of vector k onto gs_vectors[i]
            mu = inner_product(vectors[k], gs_vectors[i]) / inner_product(gs_vectors[i], gs_vectors[i])
            projection = [mu * gs_vectors[i][j] for j in range(len(gs_vectors[i]))]  # Scale gs_vectors[i] by mu
            projections[(i, k)] = projection  # Store the projection result

    return projections

In [10]:
projections = compute_projections(vectors, gs_vectors)
for (i, k), proj in projections.items():
    print(f"Projection of vector {k} with respect to vector {i}:\n", proj)

Projection of vector 1 with respect to vector 0:
 [454.0, 0.0, 0.0, 0.0, 0.0]
Projection of vector 2 with respect to vector 0:
 [1636.9999999999998, 0.0, 0.0, 0.0, 0.0]
Projection of vector 2 with respect to vector 1:
 [0.0, 107.0, 0.0, 0.0, 0.0]
Projection of vector 3 with respect to vector 0:
 [1624.0, 0.0, 0.0, 0.0, 0.0]
Projection of vector 3 with respect to vector 1:
 [0.0, 248.0, 0.0, 0.0, 0.0]
Projection of vector 3 with respect to vector 2:
 [2.6471807910693894e-12, 0.0, 1921.000000000002, 0.0, 0.0]
Projection of vector 4 with respect to vector 0:
 [1272.0, 0.0, 0.0, 0.0, 0.0]
Projection of vector 4 with respect to vector 1:
 [0.0, 1939.0, 0.0, 0.0, 0.0]
Projection of vector 4 with respect to vector 2:
 [5.911715561524058e-13, 0.0, 429.00000000000176, 0.0, 0.0]
Projection of vector 4 with respect to vector 3:
 [-9.552688178188294e-13, 0.0, 0.0, 685.9999999999983, 0.0]
